In [14]:
import getpass
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os

In [15]:
user = getpass.getuser()

nifti_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Nifti/"
nifti_paths = list(Path(nifti_dir).glob('**/*.nii'))

seg_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Segmentations/"
mask_r_paths = list(Path(seg_dir).glob('**/mask_R.nii.gz'))
mask_l_paths = list(Path(seg_dir).glob('**/mask_L.nii.gz'))

dst_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Preprocessed/"

In [21]:
def save_nib_file(path, nft, nft_affine):
    nft_2_save = nib.Nifti1Image(nft, nft_affine)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    nib.save(nft_2_save, path)


def calibrate_color_values(nft_imgs, low_cv = -1024, high_cv = 250):
    calibrated_values = nft_imgs.copy()
    for i, im in enumerate(calibrated_values):
        if any(any(c > high_cv or c < low_cv for c in r) for r in im):
            calibrated_values[i] = [[max(min(c, high_cv), low_cv) for c in r] for r in im]
    return calibrated_values

def get_z_start_stop(mask):
    start_z = 0

    for z in range(mask.shape[2]):
        if mask[:,:,z].any():
            start_z = z
            break

    stop_z = mask.shape[2]

    for z in reversed(range(mask.shape[2])):
        if mask[:,:,z].any():
            stop_z = z
            break

    return start_z, stop_z


def get_x_start_stop(mask):
    start_x = 0

    for x in range(mask.shape[0]):
        if mask[x,:,:].any():
            start_x = x
            break

    stop_x = mask.shape[0]

    for x in reversed(range(mask.shape[0])):
        if mask[x,:,:].any():
            stop_x = x
            break

    return start_x, stop_x

def generate_chunks(nft, mask_l, mask_r, dst_path, chunk_size=17, spacing=4):

    masks = [mask_l, mask_r]

    print("fetching data and calibrating values")

    calibrated_values = calibrate_color_values(nft.get_fdata())

    file_name_list = ["L", "R"]

    for i, mask in enumerate(masks):
        start_z, stop_z = get_z_start_stop(mask)
        start_x, stop_x = get_x_start_stop(mask)

        print("generating chunks for lung ", file_name_list[i])

        for z in range(start_z + chunk_size * spacing, stop_z - chunk_size*spacing, chunk_size * spacing):

            chunk = calibrated_values[start_x:stop_x, :, z : z + chunk_size]

            save_nib_file(dst_path + f"/{file_name_list[i]}_{z}_{z+chunk_size}.nii", chunk, nft.affine)    

In [22]:
def main():
    for nifti_path, mask_r_path, mask_l_path in zip(nifti_paths[:1], mask_r_paths[:1], mask_l_paths[:1]):

        nft = nib.load(nifti_path)


        mask_r = nib.load(mask_r_path).get_fdata()
        mask_l = nib.load(mask_l_path).get_fdata()

        path = dst_dir + nifti_path.stem 
        
        generate_chunks(nft, mask_l, mask_r, path)

main()

fetching data and calibrating values
generating chunks for lung  L
generating chunks for lung  R
